# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Общая информация

In [1]:
import pandas as pd

In [2]:
credit_score = pd.read_csv('/datasets/data.csv')
credit_score.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Прочитали данные с помощью метода read_csv()
Узнали общие данные info() и вывели часть таблицы для визуального анализа 

In [3]:
display(credit_score.head(13))  

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

В таблице 12 колонок и 21525 строк, содержащих в себе типы данных: 
float(дробные числа) - общий трудовой стаж в днях и ежемесячный доход, 
тип данных int(целые числа) - количество детей в семье, возраст клиента в годах, идентификатор уровня образования, идентификатор семейного положения, наличие задолженности по возврату кредитов, 
тип данных object(строковые данные) - уровень образования клиента, семейное положение, пол клиента, тип занятости, цель получения кредита. 

В столбцах days_employed и total_income есть пропущенные значения, которые являются неслучайными, так как сложно указать точный трудовой стаж в днях и кому-то может быть неловко указывать свой ежемесячный доход. Также в трудовом стаже большинство значений - отрицательные. В столбце education буквы имеют разные значения в верхнем и нижнем регистре. 

Не понимаю откуда такой высокий ежемесячный доход, да еще и у пенсионеров. Это в тенге? Или это просто ошибка, но не пойму с чем она связана. 

Это нюансы, которые сразу же бросаются в глаза. Найдем еще пропуски, ошибки и исправим их. 

## Предобработка данных

### Обработка пропусков

In [4]:
display(credit_score.isnull().sum())

credit_score['days_employed'] = credit_score['days_employed'].abs()  

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Сначала мы нашли сумму пропусков в таблице для того, чтобы понять в каких столбцах работать. 

Затем заменим пропуски в количественных переменных days_employed и total_income (заменим пропуски на медиану, так как по первым десяти строкам уже виден большой разрыв в значениях, как бы не получилось, как в анекдоте "половина людей в городе едят капусту, другая половина - мясо, но в среднем все едят голубцы").

Для того, чтобы правильно рассчитать среднее арифметическое, нужно поменять все отрицательные значения в столбце days_employed на положительные с помощью метода abs().

In [5]:
days_employed_median = credit_score.groupby(['income_type'])
credit_score['days_employed'] = days_employed_median.days_employed.apply(lambda x: x.fillna(x.median()))

total_income_median = credit_score.groupby(['income_type'])
credit_score['total_income'] = total_income_median.total_income.apply(lambda x: x.fillna(x.median()))

display(credit_score.head(13)) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


С помощью lambda функции мы заменили пропуски в столбцах по типу занятости. Сначала сгруппировали по столбцу income_type, а затем заменили пропуски в столбцах days_employed и total_income на среднее арифметическое и медиану соответственно по сгруппированному столбцу. 

Пропуск в 12 строке заменен. 

In [6]:
display(credit_score.isnull().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Проверили сумму пропусков - везде по нулям!

In [7]:
#print(credit_score['children'].value_counts())
#print(credit_score['dob_years'].value_counts())

Проверим остальные столбцы на наличие артефактов, чтобы сразу устранить ошибки. С помощью метода value_counts() были обнаружены ошибки в столбцах children и dob_years. 

В столбце с наличием детей есть значения -1 и 20. А возраст клиента в годах принимал нулевые значения. Избавимся же от этих недоразумений! 

In [8]:
credit_score['children'] = credit_score['children'].replace(-1, 1)


children_median = credit_score.loc[credit_score.loc[:, 'children'] != 20]['children'].median()
credit_score['children'] = credit_score['children'].replace(20, children_median)
 
print(credit_score['children'].value_counts())

0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


Заменяем отрицательное значение в столбце с наличием детей на положительное методом replace().

Заменяем значение 20 в том же столбце на медианное, предварительно рассчитав эту самую медиану. И прверяем данный столбец - значения -1 и 20 исчезли :) 

In [9]:
dob_years_mean = credit_score.loc[credit_score.loc[:, 'dob_years'] != 0]['dob_years'].mean()
credit_score['dob_years'] = credit_score['dob_years'].replace(0, dob_years_mean)
#print(credit_score['dob_years'].value_counts())

Ошибку в столбце про возраст заемщика исправляем заменой нулевых значений на среднее арифметическое. Проверяем - и ура! Нулевичков нет! 

**Вывод**

В столбцах с трудовым стажем и ежемесячным доходом были обнаружены пропуски, которые впоследствии мы заполнили методами mean() и median() на среднее арифметическое значение и медиану соответственно. В столбце с трудовым стажем отрицательные значения заменили на положительные, чтобы корректно можно было рассчитать среднее арифметическое значение. 

Были обнаружены ошибки в столбце с количеством детей в семье, а именно отрицательные значения -1 и 20, и в с толбце с возрастом, где появились нулевые значения. И заменены на медиану и среднее значение соответственно методом replace(). 

### Замена типа данных

In [10]:
credit_score['days_employed'] = credit_score['days_employed'].astype('int')

credit_score['total_income'] = credit_score['total_income'].astype('int')
display(credit_score.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Меняем вещественный тип данных на целочисленный методом astype(), так как именно этот метод используется для перевода значения в целое число. И проверяем 

**Вывод**

Заменили значения вещественного типа данных float на целочесленный тип int. Выбран метод astype(), так как при использовании to_numeric() все числа будут иметь тип данных float, а нам нужен int, то есть целое число.   

### Обработка дубликатов

In [11]:
print(credit_score.duplicated().sum()) 

54


In [12]:
print(credit_score['education'].value_counts()) 

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


In [13]:
credit_score['education'] = credit_score['education'].str.lower() 
print(credit_score.duplicated().sum())  

71


In [14]:
credit_score = credit_score.drop_duplicates().reset_index(drop=True) 
print(credit_score.duplicated().sum())

0


Узнаем, что в таблице 54 дубликата.  
Обнаружили в столбце одинаковые значения, но написанные в разном регистре.  
Привели уровень образования к нижнему регистру.  
Проверили количество дубликатов - их стало 71.  
Избавились от дубликатов с изменением индексов и удалением старых.

**Вывод**

Были удалены дубликаты, в том числе после того, как мы привели столбец об уровне образования к одному виду (столбец содержал в себе одинаковые значения, но написанные в разном регистре). 

### Лемматизация

In [15]:
print(credit_score['purpose'].value_counts()) 

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Возвращаем уникальные значения переменной purpose, и видим, что есть много значений, связанных с жильем, свадьбой, недвижимостью, автомобилем и образованием.

In [16]:
from pymystem3 import Mystem
m = Mystem()

In [17]:
def lemmatize_purpose(purpose):
    lemmas_row = m.lemmatize(purpose)
    for i in lemmas_row:
        if 'авто' in i:
            return 'автомобиль'
        if 'жил' in i or 'недвиж' in i:
            return 'недвижимость'
        if 'свадьб' in i:
            return 'свадьба'
        if 'образов' in i:
            return 'образование'
    

Создаем функцию, которая при обнаружении корней "авто", "жил", "недвиж", "свадьб" и "образов", будет возвращать определенные категории

In [18]:
credit_score['purpose_def'] = credit_score['purpose'].apply(lemmatize_purpose)     

#print(credit_score.head(10))

print(credit_score['purpose_def'].value_counts())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_def, dtype: int64


Создаем новый столбец с леммами и возвращаем первые десять строк таблицы для проверки. Считаем сколько раз встречается в таблице определенная категория цели кредита. 

**Вывод**

Были выделены леммы в значениях столбца с целями получения кредита - "авто", "жил", "недвиж", "свадьб" и "образов", которые вернули значения таких целей как "автомобиль", "недвижимость", "свадьба" и "образование" соответственно. В итоге мы получили 4 категории целей кредита.

### Категоризация данных

In [19]:
def income_group(income):
    if income <=150000:
        return 'низкий'
    elif 150001 <= income <= 300000:
        return "средний"
    return 'высокий'

credit_score['income_group'] = credit_score['total_income'].apply(income_group)
print(credit_score['income_group'].value_counts())  

низкий     11623
средний     8348
высокий     1483
Name: income_group, dtype: int64


Разберемся с уровнем дохода. Надо его в порядке убывания построить и узнать самые маленькие и самые большие значения, чтобы разделить на примерно пропорциональные категории:  

print(credit_score['total_income'].sort_values(by='total_income', ascending=False))

Создаем функцию, которая принимает ежемесяный доход, а возвращает уровень дохода.

Затем создаем новый столбец с уровнем дохода и добавляем туда получившиеся категории. 

**Вывод**

Для ответов на вопросы ниже нам нужны такие переменные как наличие детей, семейное положение, цели кредита и уровень дохода. Последняя переменная была категоризирована на три значения - низкий, средний и высокий уровни дохода. Семейное положение и наличие детей нет необходимости категоризировать, так как категориями как раз и будет количество вариантов в этих переменных, то есть 6 вариантов для наличия детей и 5 вариантов для семейного положения. Переменную с целями мы уже лемматизировали в прошлом разделе и соответственно разбили на 4 категории.  

## Ответы на вопросы

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [20]:
pivot_table_children = credit_score.pivot_table(index = 'children', columns = 'debt', values = 'dob_years', aggfunc = 'count')

print(pivot_table_children)

debt            0       1
children                 
0         13096.0  1071.0
1          4410.0   445.0
2          1858.0   194.0
3           303.0    27.0
4            37.0     4.0
5             9.0     NaN


Создаем сводную таблицу, в которой группируем наличие задолженности кредита по критерию наличия детей, и узнаем таким образом сколько в таблице строк с определенным количеством детей :)

In [21]:
pivot_table_common_children = pivot_table_children[1] + pivot_table_children[0]

pivot_table_children['ratio'] = pivot_table_children[1] / pivot_table_common_children * 100

print(pivot_table_children['ratio'].sort_values())

children
0    7.559822
3    8.181818
1    9.165808
2    9.454191
4    9.756098
5         NaN
Name: ratio, dtype: float64


Создаем новый столбец с процентным отношением целевого показателя к общему.

**Вывод**

Люди с 1 ребенком, с 2 и с 4 детьми чаще имеют задолженности по кредитам, чем те, у кого трое детей или вообще их нет. Можно сказать, что с увеличением количества детей в семье растет и процент невозврата кредита в срок. Но нельзя сказать однозначно, что есть зависимость между наличием детей и возвратом кредита в срок, так как разрыв в процентных соотношениях не так велик. 

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

По аналогичному принципу анализируем и следующие зависимости!

In [22]:
pivot_table_famstat = credit_score.pivot_table(index = 'family_status', columns = 'debt', values = 'dob_years', aggfunc = 'count')

print(pivot_table_famstat)

debt                       0    1
family_status                    
Не женат / не замужем   2536  274
в разводе               1110   85
вдовец / вдова           896   63
гражданский брак        3763  388
женат / замужем        11408  931


In [23]:
pivot_table_common_famstat = pivot_table_famstat[1] + pivot_table_famstat[0]

pivot_table_famstat['ratio'] = pivot_table_famstat[1] / pivot_table_common_famstat * 100

print(pivot_table_famstat['ratio'].sort_values())

family_status
вдовец / вдова           6.569343
в разводе                7.112971
женат / замужем          7.545182
гражданский брак         9.347145
Не женат / не замужем    9.750890
Name: ratio, dtype: float64


**Вывод**

В результате исследования было выявлено, что люди не обремененные браком чаще имеют задолженности по кредитам. Чаще, чем вдовы и вдовцы, и те кто в разводе. Кстати, интересная зависимость: казалось бы такие люди должны лучше просчитывать свой бюджет, у них не должно быть непредвиденных трат и трат на семью, но те кто не в браке, наоборот, не возвращают кредит в положенный срок. 

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [24]:
pivot_table_income_group = credit_score.pivot_table(index = 'income_group', columns = 'debt', values = 'dob_years', aggfunc = 'count')

print(pivot_table_income_group)

debt              0    1
income_group            
высокий        1377  106
низкий        10645  978
средний        7691  657


In [25]:
pivot_table_common_income_group = pivot_table_income_group[1] + pivot_table_income_group[0]

pivot_table_income_group['ratio'] = pivot_table_income_group[1] / pivot_table_common_income_group * 100

print(pivot_table_income_group['ratio'].sort_values())

income_group
высокий    7.147674
средний    7.870149
низкий     8.414351
Name: ratio, dtype: float64


**Вывод**

По результатам исследования было выявлено, что люди с низким уровнем дохода чаще имеют задолженности по кредитам, чем люди с высоким и средним уровнями дохода. Однако банки все равно не смогут одобрить большой кредит, не удостоверившись, что доход соответствует погашению. 

- **Как разные цели кредита влияют на его возврат в срок?**

In [26]:
pivot_table_purpose_def = credit_score.pivot_table(index = 'purpose_def', columns = 'debt', values = 'dob_years', aggfunc = 'count')

print(pivot_table_purpose_def)

debt              0    1
purpose_def             
автомобиль     3903  403
недвижимость  10029  782
образование    3643  370
свадьба        2138  186


In [27]:
pivot_table_common_purpose_def = pivot_table_purpose_def[1] + pivot_table_purpose_def[0]

pivot_table_purpose_def['ratio'] = pivot_table_purpose_def[1] / pivot_table_common_purpose_def * 100

print(pivot_table_purpose_def['ratio'].sort_values())

purpose_def
недвижимость    7.233373
свадьба         8.003442
образование     9.220035
автомобиль      9.359034
Name: ratio, dtype: float64


**Вывод**

Задолженности случаются чаще у тех, кто брал кредит для автомобиля, образования и всех связанных с этим целей. Реже имеют задолженность по кредиту те, кто брал кредит на дела с недвижимостью и на свадьбу. 

In [36]:
pivot_table_purpose_income = credit_score.pivot_table(index = ['income_group','purpose_def'], columns = 'debt', values = 'dob_years', aggfunc = 'count')

pivot_table_purpose_income['ratio'] = pivot_table_purpose_income[1] / (pivot_table_purpose_income[0] + pivot_table_purpose_income[1])* 100

print(pivot_table_purpose_income['ratio'])

income_group  purpose_def 
высокий       автомобиль      7.920792
              недвижимость    7.335907
              образование     6.349206
              свадьба         5.960265
низкий        автомобиль      9.554140
              недвижимость    7.276507
              образование     9.577338
              свадьба         9.433962
средний       автомобиль      9.344660
              недвижимость    7.156265
              образование     9.173715
              свадьба         6.326304
Name: ratio, dtype: float64


## Общий вывод

В процессе исследования надежности заемщиков, были изучены и обработаны полученные данные. Определены пропущенные значения, ошибки, обработаны дубликаты, данные в столбцах приведены к общему виду, была произведена лемматизация и категоризация значений для ответа на поставленные вопросы о зависимости между определенными переменными и возвратом кредита в срок. 

В результате исследования надежности заемщиков мы пришли к выводу, что при выдаче кредита лучше обращать внимание на семейное положение, цель кредита и наличие детей. Уровень дохода и так обрабатывается при заявке на выдачу кредита, хотя по результатам исследования было выявлено, что люди с низким уровнем дохода чаще имеют задолженность по кредитам. Банк не выдаст кредит тому, кто объективно по уровню дохода не сможет его вовремя погасить.